In [1]:
import numpy as np
import pandas as pd
import imageio

import plotly.offline as off
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import plotting_utils as pu
import metrics as mtc

import warnings
warnings.simplefilter("ignore")

In [2]:
# Specify data file and load both dataframes from it
datafile = '/data/RocklandSample/derivatives/mca/mca-onevox-fullrr.h5'
df = pd.read_hdf(datafile, 'graphs')

In [3]:
sub = df['sub'].value_counts().to_dict()
df_ = df.loc[df['noise_type'] == df['noise_type']]
_df = df.loc[df['noise_type'] != df['noise_type']].loc[df["os"] == "ubuntu"]

In [4]:
met = 'fro'
distance = '{0} (percent difference)'.format(met)
log10_status = True

for ss in sub:
    ref = _df.loc[df['sub'] == ss].iloc[0]
    idx = df_.loc[df['sub'] == ss].index
    df_.loc[idx, distance] = 100.0 * df_.loc[idx, '{0} (ref)'.format(met)] / ref['{0} (self)'.format(met)]

In [12]:
# Get distribution of subject-level noise
shp, anno = pu.create_bounding_box(_df, "graph", met, df2=None, percentage=True, log10=True,
                                   color="LightSalmon", title="X-Subject")

# Get distribution of OS-level noise
_df_ub = df.loc[df['noise_type'] != df['noise_type']].loc[df["os"] == "ubuntu"]
_df_al = df.loc[df['noise_type'] != df['noise_type']].loc[df["os"] == "alpine"]
shp2, anno2 = pu.create_bounding_box(_df_ub, "graph", met, df2=_df_al,
                                     df2_query="sub == '{0}'", df2_query_cols=['sub'],
                                     percentage=True, log10=True, color="gray", title="X-OS")
shapes = shp + shp2
annos = anno + anno2

In [13]:
# Specify some plot features
png_outpath = '/data/RocklandSample/derivatives/mca/pngs/{0}_summary.png'.format(met)

cord = {"noise_type": ['python_mca', 'python_rr', 'full_rr', 'independent', 'single']}
title = "Differences in Monte Carlo Arithmetic Injected Structural Connectomes"

if log10_status:
    y_column = "log {0}".format(distance)
    df_[y_column] = np.log10(df_[distance].astype(float) + 1)
else:
    y_column = distances
    
tickvals, ticktext = set_ticks(upper=df_[distance].max(), log10=log10_status)

In [14]:
fig = px.box(df_, y=y_column, x='sub', color='noise_type',
             points='all', notched=False, boxmode="group",
             hover_data=['simulation_id'], category_orders=cord)

fig.update_layout(yaxis={"tickvals": tickvals,
                         "ticktext": ticktext,
                         "title": "{0} (% Deviation)".format(mtc.met_names[met])},
                  xaxis={"title": "Subject"},
                  shapes=shapes, annotations=annos,
                  title=title)

fig.update_layout(height=750, width=1350, font=dict(size=20))

# fig.update_layout(shapes=shapes, annotations=annos,
#                   legend={'y':0.9, 'yanchor': 'top',
#                           'x':0.9, 'xanchor': 'right'})

fig.show()
# fig.write_image(png_outpath)
# fig.write_image(png_outpath.replace('.png', '.pdf'))
# off.plot(fig, filename=png_outpath.replace('.png', '.html'))

# fig = px.violin(df_, y='log {0}'.format(distance), x='sub', color='noise_type',
#                 points='outliers', box=False, violinmode='group', category_orders=cord,
#                 hover_data=['simulation_id'], title=title)

In [47]:
# The below values were picked for the Frobenius norm
# ss = 'A00035940'
ss = 'A00057725'

df_ss = df_.query('sub == "{0}" and noise_type == "rr"'.format(ss))
modes = df_ss[distance].value_counts().sort_index().to_dict()
mkeys = sorted(modes.keys())

keylocs = [1, 2, 4, 5, 0, 8, 10, 11, 16]

refvol = 4

graphs_to_plot = []
subplot_titles = []
for k in keylocs:
    loc = mkeys[k]
    graphs_to_plot += [df_ss[df_ss[distance] == loc].iloc[0]['graph']]
    if k:
        subplot_titles += ["%d / 100 (%.3f %%)" % (modes[loc], loc)]
    else:
        subplot_titles += ["reference"]

In [10]:
png_outpath = '/data/RocklandSample/derivatives/mca/pngs/{0}_{1}'.format(met, ss)

axis_template = dict(showgrid = False, zeroline = False,
                     showticklabels = False, ticks = '')

ref_template = dict(mirror=True, showline=True,
                    linewidth=6, linecolor='orange')

fig = make_subplots(rows=3, cols=3,
                    vertical_spacing=0.05,
                    horizontal_spacing=0.03,
                    subplot_titles=subplot_titles)

for idx, gtp in enumerate(graphs_to_plot):
    r, c = int((idx)/ 3) + 1, int((idx) % 3)+1 
    modif = '' if not idx else str(idx + 1)
    if idx == refvol:
        dat = np.log10(gtp+1)
    else:
        dat = gtp - graphs_to_plot[refvol]
        dat = np.sign(dat)*np.log10(np.abs(dat) + 1)

    trace = go.Heatmap(z=dat, coloraxis="coloraxis")
    fig.append_trace(trace, row=r, col=c)
    for k in axis_template:
        fig['layout']['xaxis'+modif][k] = axis_template[k]
        fig['layout']['yaxis'+modif][k] = axis_template[k]
    
    if idx == refvol:
        for k in ref_template:
            fig['layout']['xaxis'+modif][k] = ref_template[k]
            fig['layout']['yaxis'+modif][k] = ref_template[k]

    fig['layout']['yaxis'+modif]['autorange'] = "reversed"

title = 'Error-Induced Deviations from Reference Connectome (Subject {0})'.format(ss)
fig.update_coloraxes(dict(colorscale=px.colors.diverging.RdBu[::-1],
                          showscale=True, cmin=0, cmid=1.5, cmax=3,
                          colorbar=dict(tickvals=[0, 3],
                                        title="Connectivity Strength",
                                        ticktext=['no difference',
                                                  'stronger'])))

fig.update_layout(title=title, height=1500, width=1600, font=dict(size=26))
for idx, anno in enumerate(fig.layout.annotations):
    fig.layout.annotations[idx]['font']['size'] = 28
fig.show()
fig.write_image(png_outpath + '_panel.png')

In [11]:
png_outpath = '/data/RocklandSample/derivatives/mca/pngs/{0}'.format(ss)

axis_template = dict(showgrid = False, zeroline = False,
                     showticklabels = False, ticks = '',
                     autorange="reversed")

df_ss = df_.query('sub == "{0}" and noise_type == "rr"'.format(ss))
modes = df_ss[distance].value_counts().sort_index().to_dict()
mkeys = sorted(modes.keys())

pngs = []
for idx, k in enumerate(mkeys):
    g = df_ss[df_ss[distance] == k].iloc[0]['graph']
    t = "Difference from Reference: %.3f %%" % (k)

    dat = np.log10(g+1)
    trace = go.Heatmap(z=dat, colorscale=px.colors.diverging.RdBu[::-1],
                       showscale=False, zmin=0, zmid=1.75, zmax=3.5)
    
    single = go.Figure(data=trace)
    title = [go.layout.Annotation(x=0.5, y=1.1, font={"size":22},
                                  xref="paper", yref="paper",
                                  text=t, ax=0, ay=0)]
    single.update_layout(annotations = title, yaxis=axis_template)

    fname = png_outpath + "_{0}.png".format(idx)
    pngs += [fname]
    single.write_image(fname, format='png')

In [42]:
images = []
for filename in pngs:
    images.append(imageio.imread(filename))
imageio.mimsave(png_outpath + ".gif", images, duration=0.3)